## Initialize

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

## Import the Data

In [3]:
dataset = pd.read_csv("Car.data", header = None)
dataset.shape

(1728, 7)

In [4]:
dataset.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


## Create X and Y

In [5]:
X = dataset.iloc[:, 0:6].values
Y = dataset.iloc[:, 6].values

In [6]:
X.shape

(1728, 6)

In [7]:
Y.shape

(1728,)

## Preprocess the Data

In [8]:
le_Y = LabelEncoder()
Y = le_Y.fit_transform(Y)
Y = Y.reshape(len(Y), 1)

def encoder(index):
    le_X = LabelEncoder()
    X[:, index] = le_X.fit_transform(X[:, index])

In [9]:
for i in range(0, 6):
    encoder(i)

In [10]:
X[0]

array([3, 3, 0, 0, 2, 1], dtype=object)

In [11]:
Y[0]

array([2], dtype=int64)

In [12]:
ohe_Y = OneHotEncoder(categorical_features = [0])
Y = ohe_Y.fit_transform(Y).toarray()

In [13]:
Y[0]

array([ 0.,  0.,  1.,  0.])

In [14]:
Y.shape

(1728, 4)

In [15]:
ohe_X = OneHotEncoder(categorical_features = [5])
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]

In [16]:
ohe_X = OneHotEncoder(categorical_features = [6])
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]

In [17]:
ohe_X = OneHotEncoder(categorical_features = [7])
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]

In [18]:
ohe_X = OneHotEncoder(categorical_features = [8])
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]

In [19]:
ohe_X = OneHotEncoder(categorical_features = [10])
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]

In [20]:
ohe_X = OneHotEncoder(categorical_features = [12])
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]

In [21]:
X.shape

(1728, 15)

## Create Train and Test Data

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [23]:
X_train.shape

(1382, 15)

In [24]:
Y_train.shape

(1382, 4)

In [25]:
X_test.shape

(346, 15)

In [26]:
Y_test.shape

(346, 4)

## Create TF Classifier

In [27]:
num_features = X.shape[1]
num_features

15

In [28]:
num_classes = Y.shape[1]
num_classes

4

In [29]:
'''
Y = W1.X1 + ... + W4.X4 + B
output = softmax(Y)
Create Weights (W) and Biases(B) variables
'''

W = tf.Variable(tf.zeros([num_features, num_classes]))
B = tf.Variable(tf.zeros([num_classes]))

In [30]:
# Create x and y_ placeholders for actual data
x = tf.placeholder(tf.float32, [None, num_features])
y_ = tf.placeholder(tf.float32, [None, num_classes])

In [31]:
# Calculate y which holds model's predicted values
Wx = tf.matmul(x, W)
y = Wx + B

In [32]:
# Create cost function to minimize
cost_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y))

In [33]:
# Create an optimizer to minimize the cost function
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(cost_cross_entropy)

In [34]:
def trainTheData(num_steps, optimizer_to_use, batch_size):
    init = tf.global_variables_initializer()
    # initialize the global variables
    
    with tf.Session() as sess:
        sess.run(init)
        
        for i in range(num_steps):
            
            # Calculate the batch start index
            if batch_size == len(X_train):
                batch_start_index = 0
            elif batch_size > len(X_train):
                raise ValueError("Batch Size " + str(batch_size) + " cannot be greater than Data Size : ", len(X_train))
            else:
                batch_start_index = (i * batch_size) % (len(X_train) - batch_size)
            
            # Calculate the batch end index
            batch_end_index = batch_size + batch_start_index
            
            # Create batch x and y values
            batch_X_values = X_train[batch_start_index : batch_end_index]
            batch_Y_values = Y_train[batch_start_index : batch_end_index]
            
            # Create feed dict to feed it to optimizer
            feed = {x : np.array(batch_X_values), y_ : np.array(batch_Y_values)}
            
            sess.run(optimizer_to_use, feed_dict = feed)
            
            # Print the metrics for every 50th iteration
            if (i + 1) % 50 == 0:
                print("After " + str(i + 1) + " iterations, cost : ", sess.run(cost_cross_entropy, feed_dict = feed))
                print("W : ", sess.run(W))
                print("B : ", sess.run(B))
                print("")
                
        Y_pred_logits = sess.run(y, feed_dict = {x : np.array(X_test)})
        
        Y_pred_logits_tensor = tf.convert_to_tensor(np.array(Y_pred_logits))
        
        apply_softmax = tf.nn.softmax(Y_pred_logits_tensor)
        
        Y_pred_classes = np.argmax(sess.run(apply_softmax), axis = 1)
        
        sess.close()
    return Y_pred_classes

In [35]:
Y_pred_classes = trainTheData(num_steps = 2000, optimizer_to_use = optimizer, batch_size = len(X_train))

After 50 iterations, cost :  0.555137
W :  [[-0.12054089  0.16993222 -0.21289603  0.1635047 ]
 [ 0.10515174 -0.04629518 -0.06407927  0.00522265]
 [-0.05394488 -0.30966461  0.66001606 -0.29640645]
 [-0.13551946  0.21652766 -0.08500708  0.00399888]
 [ 0.16856034 -0.07671861 -0.08905069 -0.00279105]
 [-0.05841188 -0.30930397  0.66887826 -0.30116233]
 [ 0.08535282 -0.17291644  0.19142993 -0.10386629]
 [ 0.09759738 -0.1168248   0.08319436 -0.06396692]
 [ 0.08118141 -0.10549241  0.09073673 -0.06642575]
 [ 0.7215746  -0.00415616 -0.69476813 -0.0226503 ]
 [ 0.64680195 -0.06845995 -0.58213419  0.00379212]
 [ 0.01110059 -0.17340706  0.25999346 -0.09768683]
 [-0.02208058 -0.15811825  0.59102446 -0.41082546]
 [-0.80963385 -0.3628827   1.53979838 -0.36728144]
 [ 0.24200971  0.00333356  0.24829529 -0.49363837]]
B :  [ 0.06518616 -0.62926972  1.10910404 -0.54502237]

After 100 iterations, cost :  0.467821
W :  [[-0.29153994  0.35601372 -0.42852336  0.3640494 ]
 [ 0.05765679  0.01890956 -0.19718444  0

After 550 iterations, cost :  0.303208
W :  [[ -7.63854742e-01   1.00565386e+00  -1.35169220e+00   1.10989320e+00]
 [ -2.96961248e-01   5.05657256e-01  -9.74109054e-01   7.65412331e-01]
 [  2.69581676e-01  -9.13511336e-01   1.53120279e+00  -8.87271762e-01]
 [ -6.36965930e-01   1.19266546e+00  -9.02527630e-01   3.46827298e-01]
 [ -6.17438033e-02   4.75017846e-01  -8.45114112e-01   4.31840271e-01]
 [  3.35646749e-01  -9.17103171e-01   1.65778089e+00  -1.07632422e+00]
 [  2.63307393e-01  -3.59715104e-01   4.01547225e-03   9.23921540e-02]
 [  2.05804706e-01  -1.31880865e-01  -2.94182479e-01   2.20258653e-01]
 [  1.64760053e-01  -9.46462452e-02  -2.76334226e-01   2.06220493e-01]
 [  1.98689163e+00   4.73177314e-01  -3.10029769e+00   6.40231788e-01]
 [  1.91464484e+00   3.09587449e-01  -2.95906973e+00   7.34836042e-01]
 [ -6.53431192e-02  -2.35448301e-01   4.99618620e-01  -1.98827013e-01]
 [  1.68015733e-01  -4.01221216e-02   1.55192816e+00  -1.67982042e+00]
 [ -2.03597713e+00  -9.84499991e-

After 1050 iterations, cost :  0.259695
W :  [[-0.94295985  1.35686255 -1.95616269  1.54226148]
 [-0.46065134  0.83424324 -1.49980676  1.12621403]
 [ 0.45590973 -1.11559629  1.78336811 -1.12368095]
 [-0.74366307  1.58335507 -1.31123006  0.47153872]
 [-0.15745914  0.85788143 -1.24404204  0.54362005]
 [ 0.57393974 -1.12021124  2.03678274 -1.49051166]
 [ 0.28049636 -0.36277965 -0.11755072  0.19983383]
 [ 0.19922963 -0.08488213 -0.49944645  0.38509905]
 [ 0.15606831 -0.04910581 -0.47162914  0.36466688]
 [ 2.33106494  0.71027684 -4.0998311   1.05849111]
 [ 2.25681829  0.53603011 -3.95216942  1.15932345]
 [-0.04430943 -0.21428956  0.60966629 -0.35106814]
 [ 0.32975161  0.01785103  2.00729251 -2.35488963]
 [-2.32998633 -1.27027249  5.39974451 -1.79948997]
 [ 0.57402271  0.60289818  1.59452999 -2.77145052]]
B :  [-0.42556646 -2.49579477  4.52388239 -1.60251975]

After 1100 iterations, cost :  0.256881
W :  [[-0.95748663  1.3854357  -2.00616837  1.57822132]
 [-0.47322339  0.86069924 -1.54256141

After 1550 iterations, cost :  0.237378
W :  [[ -1.07614493e+00   1.61105514e+00  -2.40180707e+00   1.86689544e+00]
 [ -5.73255718e-01   1.06751812e+00  -1.87764084e+00   1.38337910e+00]
 [  5.74706495e-01  -1.23415267e+00   1.93437588e+00  -1.27492917e+00]
 [ -8.25185359e-01   1.86475432e+00  -1.60998416e+00   5.70412934e-01]
 [ -2.27570951e-01   1.12835181e+00  -1.52413011e+00   6.23350024e-01]
 [  7.35557020e-01  -1.23824430e+00   2.29479456e+00  -1.79210567e+00]
 [  2.72677660e-01  -3.37230295e-01  -2.09223688e-01   2.73776084e-01]
 [  1.73188865e-01  -3.13716903e-02  -6.51614130e-01   5.09796858e-01]
 [  1.28135353e-01   1.37822970e-03  -6.16593361e-01   4.87079799e-01]
 [  2.53796721e+00   8.94801557e-01  -4.81637573e+00   1.38360965e+00]
 [  2.46131468e+00   7.13362217e-01  -4.66462755e+00   1.48994946e+00]
 [ -2.65807770e-02  -2.06480414e-01   6.86183095e-01  -4.53122973e-01]
 [  4.56554443e-01   3.44143771e-02   2.33332968e+00  -2.82429147e+00]
 [ -2.49136782e+00  -1.48481691e

In [36]:
Y_test_classes = np.argmax(Y_test, axis = 1)

## Calculate the Accuracy

In [37]:
accuracy_score(Y_test_classes, Y_pred_classes)

0.91907514450867056

In [38]:
confusion_matrix(Y_test_classes, Y_pred_classes)

array([[ 73,   2,   8,   0],
       [  6,   4,   0,   3],
       [  8,   0, 232,   1],
       [  0,   0,   0,   9]], dtype=int64)